# Introduction

This project will explore the OCCP data. Open Charge Point Protocol (OCPP) is an open standard communication protocol for Electric Vehicle (EV) charging stations. It defines interactions between EV charging stations and a central system, helping to facilitate security, transactions, diagnostics, and more.

This dataset if from OCCP v1.6

## Charging System Diagram
Organization < Property < Location < Cluster < Station < UserID

A cluster is a grouping of chargers/stations. This for convenience/load balancing

Each circuit can have multiple clusters.

Each cluster has its own breaker


## Prepare Enviornment

In [1]:
# Access to Google Drive
# This seems to propagate credentials better from its own cell

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Packages and methods

!pip install PyGithub
from github import Github
import os
import datetime
from google.colab import userdata

!pip install pandas pyxlsb
import pandas as pd

import numpy as np

import sys
import logging
import psycopg2

!pip install SQLAlchemy psycopg2-binary
import seaborn as sns
import json

import statsmodels.api as sm
from statsmodels.formula.api import ols

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import matplotlib.pyplot as plt

from datetime import timedelta
import holidays

!pip install statsmodels
import statsmodels.api as sm


In [2]:
# Update github

def colab_to_github(notebook_path, github_repo, folder_path=None, commit_message=None, branch="main"):
   try:
       print("Fetching GitHub token...")
       token = os.getenv('GITHUB_TOKEN')
       if not token:
           raise ValueError("GitHub token is missing or invalid. Ensure it is set as an environment variable.")

       # Add debug logging (only showing first few chars for security)
       print(f"Token format check - starts with: {token[:4]}")

       print("Token successfully retrieved.")
       g = Github(token)
       repo = g.get_repo(github_repo)
       print(f"Connected to repository: {github_repo}")

       if not commit_message:
           commit_message = f"Auto-commit from Colab: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
       print(f"Using commit message: {commit_message}")

       with open(notebook_path, 'r') as file:
           notebook_content = file.read()
       print(f"Notebook content read from {notebook_path}")

       filename = os.path.basename(notebook_path)
       # Construct the full file path including the folder if specified
       file_path = f"{folder_path}/{filename}" if folder_path else filename
       print(f"Target file path in repo: {file_path}")

       try:
           print(f"Checking if file exists at {file_path}...")
           existing_file = repo.get_contents(file_path, ref=branch)
           repo.update_file(
               path=file_path,
               message=commit_message,
               content=notebook_content,
               sha=existing_file.sha,
               branch=branch
           )
           print(f"File updated successfully in branch '{branch}'.")
       except Exception:
           print(f"File does not exist at {file_path}. Attempting to create...")
           repo.create_file(
               path=file_path,
               message=commit_message,
               content=notebook_content,
               branch=branch
           )
           print(f"File created successfully in branch '{branch}'.")

   except Exception as e:
       print(f"Error occurred: {e}")

raw_token = userdata.get('GITHUB_TOKEN')
cleaned_token = raw_token.replace('token ', '').strip()
print(f"Cleaned token starts with: {cleaned_token[:4]}")

os.environ['GITHUB_TOKEN'] = cleaned_token

# Call the function with your parameters
notebook_path = "/content/drive/MyDrive/Colab Notebooks/OCCP.ipynb"
github_repo = "davidelgas/DataSciencePortfolio"  # This is the correct repository path
folder_path = "OCCP"  # This specifies the directory within the repository
commit_message = "Updated notebook from Colab"

colab_to_github(notebook_path, github_repo, folder_path, commit_message)

Cleaned token starts with: ghp_
Fetching GitHub token...
Token format check - starts with: ghp_
Token successfully retrieved.
Connected to repository: davidelgas/DataSciencePortfolio
Using commit message: Updated notebook from Colab
Notebook content read from /content/drive/MyDrive/Colab Notebooks/OCCP.ipynb
Target file path in repo: OCCP/OCCP.ipynb
Checking if file exists at OCCP/OCCP.ipynb...
File updated successfully in branch 'main'.


## Ingest raw data

In [ ]:
# import log data

import pandas as pd
import numpy as np

def load_file(file_path):
    """Load a single CSV file."""
    return pd.read_csv(file_path)

def concatenate_files(file_paths):
    """Load and combine multiple CSV files."""
    dfs = []
    for file_path in file_paths:
        df = load_file(file_path)
        if not df.empty:
            dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

if __name__ == "__main__":
    file_paths = [
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/sept_100_sample.csv',
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/oct_100_sample.csv',
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/nov_100_sample.csv',
        '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/dec_100_sample.csv'
    ]

    # Concatenate all files
    df_logs = concatenate_files(file_paths)

    # Save the combined raw data
    df_logs.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs.pkl')

In [ ]:
#Import property tables from AWS

import psycopg2
import pandas as pd
import os

# Load credentials
def load_credentials(path_to_credentials):
    with open(path_to_credentials, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

# Connection parameters
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Connect and fetch data
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Fetch properties table
cursor.execute("SELECT * FROM properties;")
df_prop = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df_prop.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.csv', index=False)
df_prop.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.pkl')

# Fetch property_types table
cursor.execute("SELECT * FROM property_types;")
df_prop_type = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df_prop_type.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.csv', index=False)
df_prop_type.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.pkl')

# Close connection
cursor.close()
connection.close()

In [ ]:
#Import property table from SalesForce

import pandas as pd
from pathlib import Path
import os

def process_property_sizes(input_file: Path) -> pd.DataFrame:
    # Define fields to cast as int
    int_fields = [
        'account_id',
        'managed_account_id',
        'Parking Space Count'
        # Add any other fields that should be int
    ]

    # Load CSV with explicit encoding
    df_prop_size = pd.read_csv(input_file, encoding='latin-1')

    # Cast specified fields to int, handling any errors
    for field in int_fields:
        if field in df_prop_size.columns:
            df_prop_size[field] = pd.to_numeric(df_prop_size[field], errors='coerce').fillna(0).astype(int)

    # Save pickle to same directory as input file
    prop_size_pickle_path = input_file.parent / 'df_prop_size.pkl'
    df_prop_size.to_pickle(prop_size_pickle_path)

    # Print info about the conversions
    print("\nData types after conversion:")
    print(df_prop_size[int_fields].dtypes)

    return df_prop_size

if __name__ == "__main__":
    # Define base directory
    data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')

    # Input path
    prop_size_input = data_dir / 'Properties Table jan2025.csv'

    # Run workflow
    df_prop_size = process_property_sizes(prop_size_input)


Data types after conversion:
account_id             int64
managed_account_id     int64
Parking Space Count    int64
dtype: object


## Clean data

In [4]:
# Here are the dfs Ill be working with

df_logs = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs.pkl') # Event data from Splunk
df_prop = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop.pkl') # Property metadata from AWS
df_prop_size = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_size.pkl') # Property size data from SalesForce
df_prop_type = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_prop_type.pkl') # Property metadata from AWS

###This is what the JSON field looks like



{"connectorId":1,"transactionId":1417592169,"meterValue":[{"timestamp":"2025-01-14T13:27:37.145Z","sampledValue":[{"value":"31323855.0","context":"Sample.Periodic","format":"Raw","measurand":"Energy.Active.Import.Register","phase":"L1","location":"Outlet","unit":"Wh"},{"value":"0.00","context":"Sample.Periodic","format":"Raw","measurand":"Current.Import","phase":"L1","location":"Outlet","unit":"A"},{"value":"240.57","context":"Sample.Periodic","format":"Raw","measurand":"Voltage","phase":"L1","location":"Outlet","unit":"V"},{"value":"28","context":"Sample.Periodic","format":"Raw","measurand":"Temperature","phase":null,"location":"Body","unit":"Celsius"},{"value":"6.00","context":"Sample.Periodic","format":"Raw","measurand":"Current.Offered","phase":"L1","location":"Outlet","unit":"A"},{"value":"1440.00","context":"Sample.Periodic","format":"Raw","measurand":"Power.Offered","phase":"L1","location":"Outlet","unit":"W"},{"value":"0.00","context":"Sample.Periodic","format":"Raw","measurand":"Power.Active.Import","phase":"L1","location":"Outlet","unit":"W"}]}]}

In [5]:
# Unpack the JSON field in the log file
import pandas as pd
import json

def expand_message_json(df):
    rows = []

    for idx, row in df.iterrows():
        # Parse the JSON message
        message = json.loads(row['cleaned_message']) if pd.notna(row['cleaned_message']) else {}

        # Get transactionId from the message
        transaction_id = message.get('transactionId')

        # Extract meter values
        meter_values = message.get('meterValue', [])
        for meter in meter_values:
            timestamp = meter.get('timestamp')
            sampled_values = meter.get('sampledValue', [])

            # Filter for only A and W units
            for sample in sampled_values:
                unit = sample.get('unit')
                if unit in ['A', 'W']:
                    rows.append({
                        'property_id': row['property_id'],
                        'user_id': row['user_id'],
                        'transaction_id': transaction_id,  # Fixed variable name here
                        'timestamp': timestamp,
                        'value': sample.get('value'),
                        'unit': unit
                    })

    return pd.DataFrame(rows)

# Expand the JSON column
df_logs_exp = expand_message_json(df_logs)

# Save the expanded data
df_logs_exp.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_exp.pkl')

# Expand the JSON column
df_logs_exp = expand_message_json(df_logs)

# Save the expanded data
df_logs_exp.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_exp.pkl')


In [6]:
# Create a sample of log data

import pandas as pd
from pathlib import Path

# Load the DataFrame
data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')
logs_pickle = data_dir / 'df_logs_exp.pkl'

# Read pickle and sample 10 records
df_logs_exp = pd.read_pickle(logs_pickle)
sample_df = df_logs_exp.sample(n=10, random_state=42)  # random_state for reproducibility

# Save sample to CSV in same directory
sample_df.to_csv(data_dir / 'logs_sample.csv', index=False)

print("10 record sample saved to logs_sample.csv")

10 record sample saved to logs_sample.csv


In [4]:
# Fix field name and datatype inconsistencies
import pandas as pd
from pathlib import Path

def normalize_timestamp(df_path: Path) -> None:
   # Load DataFrame
   df = pd.read_pickle(df_path)

   # Convert to datetime and truncate to seconds
   df['_time'] = pd.to_datetime(df['_time']).dt.floor('s')

   # Save back to pickle
   df.to_pickle(df_path)

   print("Timestamps normalized and saved")

def normalize_account_ids(df_path: Path) -> None:
   # Load DataFrame
   df = pd.read_pickle(df_path)

   # Cast ID fields to int
   df['managed_account_id'] = pd.to_numeric(df['managed_account_id'], errors='coerce').fillna(0).astype(int)
   df['account_id'] = pd.to_numeric(df['account_id'], errors='coerce').fillna(0).astype(int)

   # Save back to pickle
   df.to_pickle(df_path)

   print("Account IDs normalized and saved")

def normalize_prop_size_id(df_path: Path) -> None:
   # Load DataFrame
   df = pd.read_pickle(df_path)

   # Cast ID field to int
   df['managed_account_id'] = pd.to_numeric(df['managed_account_id'], errors='coerce').fillna(0).astype(int)

   # Save back to pickle
   df.to_pickle(df_path)

   print("Property size ID normalized and saved")

if __name__ == "__main__":
   # Define base directory
   data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')

   # Define pickle files
   logs_pickle = data_dir / 'df_logs.pkl'
   prop_pickle = data_dir / 'df_prop.pkl'
   prop_size_pickle = data_dir / 'df_prop_size.pkl'

   # Run normalizations
   normalize_timestamp(logs_pickle)
   normalize_account_ids(prop_pickle)
   normalize_prop_size_id(prop_size_pickle)

Timestamps normalized and saved
Account IDs normalized and saved
Property size ID normalized and saved


In [5]:
# Check join logic
# Does df_logs.property_ud join with df_prop.id ?
# Sample value 3436570000094511023
# Found 20 property_ids in logs that don't exist in properties table

import pandas as pd
from pathlib import Path

# Load the DataFrames
data_dir = Path('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP')
logs_pickle = data_dir / 'df_logs.pkl'
prop_pickle = data_dir / 'df_prop.pkl'

df_logs = pd.read_pickle(logs_pickle)
df_prop = pd.read_pickle(prop_pickle)

# Check unique values in each column
n_log_props = df_logs['property_id'].nunique()
n_prop_ids = df_prop['id'].nunique()

print(f"Unique property_ids in logs: {n_log_props}")
print(f"Unique ids in properties: {n_prop_ids}")

# Check which property_ids in logs don't exist in properties
missing_props = set(df_logs['property_id'].unique()) - set(df_prop['id'].unique())

if missing_props:
    print(f"\nFound {len(missing_props)} property_ids in logs that don't exist in properties table")
    print("Sample of missing ids:", list(missing_props)[:5])
else:
    print("\nAll property_ids in logs exist in properties table")

# Check actual join
merged_df = df_logs.merge(df_prop, left_on='property_id', right_on='id', how='left')
n_unmatched = merged_df['id'].isna().sum()

print(f"\nUnmatched rows after join: {n_unmatched} ({(n_unmatched/len(df_logs))*100:.2f}% of logs)")


Unique property_ids in logs: 700
Unique ids in properties: 807

Found 20 property_ids in logs that don't exist in properties table
Sample of missing ids: [nan, '53e9cab2-aed2-4c35-ab39-e8375825e6fb\n4cb8af8c-e0ed-41d9-8c08-6e66c4125cdd', 'f14f0f6a-1ec0-4136-8bf4-90b96ec0f604\n0b372bad-6c5f-4ff7-ab2f-5f9b380d5e6f', '500c202d-143e-4e3d-854c-77fefb209253\n406ec42d-df0b-401d-a0a2-5e21824cb3d0', 'e95523e6-3470-4a60-b586-ee715cd9f34b\n9a06f545-1ab0-476d-8d43-1b5101cbd551']

Unmatched rows after join: 8054 (0.45% of logs)


In [6]:
# Sample missing data

import psycopg2
import pandas as pd
import os

# Load credentials
def load_credentials(path_to_credentials):
   with open(path_to_credentials, 'r') as file:
       for line in file:
           if '=' in line:
               key, value = line.split('=', 1)
               os.environ[key.strip()] = value.strip()

# Connection parameters
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

connection_params = {
   'host': os.getenv('DB_HOST'),
   'dbname': os.getenv('DB_NAME'),
   'user': os.getenv('DB_USER'),
   'password': os.getenv('DB_PASSWORD'),
   'port': os.getenv('DB_PORT')
}

# Connect and fetch data
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Fetch full rows where property_type is NULL
query = "SELECT * FROM properties WHERE property_type IS NULL;"

cursor.execute(query)
columns = [desc[0] for desc in cursor.description]
missing_prop_types = pd.DataFrame(cursor.fetchall(), columns=columns)

print(missing_prop_types.to_string())

# Close connection
cursor.close()
connection.close()

                                     id                       organization_id                                             name       phone contact            longitude            latitude  watts_soft_limit property_type  note   utility_provider gateway_type carrier_name               address_1  address_2         city state    zip email  has_editable_penalty   managed_account_id                       created_at                       updated_at reboot_cron_schedule account_id  hidden decommissioned_at exclusive_domain_1 exclusive_domain_2
0  522f3cfd-1a6a-4929-be9e-793642a7d1cd  e4c8d63e-b801-4d72-830a-0a5a8aad8097                    TEST - NewPropertyTestPricing                                     None                None               100          None                    LADWP         None         None                                                                                    False                      2024-10-02 04:45:34.289552+00:00 2025-01-08 21:12:29.442263+00:00           

In [7]:
# Check join logic
# Does df_logs.property_id join with df_prop_size.id

# Check join keys
print("Unique property_id in df_logs:", df_logs['property_id'].nunique())
print("Unique ids in df_prop_size:", df_prop_size['id'].nunique())

# Check overlap between keys
common_keys = set(df_logs['property_id']) & set(df_prop_size['id'])
print("\nNumber of common property IDs:", len(common_keys))

# Percentage of matching keys
logs_match_percentage = len(common_keys) / df_logs['property_id'].nunique() * 100
prop_size_match_percentage = len(common_keys) / df_prop_size['id'].nunique() * 100

print(f"\nPercentage of property_id in logs that match prop_size: {logs_match_percentage:.2f}%")
print(f"Percentage of ids in prop_size that match logs: {prop_size_match_percentage:.2f}%")

# Sample of matching and non-matching keys
print("\nSample of matching property IDs (first 10):")
print(list(common_keys)[:10])

# Check for any non-matching keys
non_matching_in_logs = set(df_logs['property_id']) - set(df_prop_size['id'])
non_matching_in_prop_size = set(df_prop_size['id']) - set(df_logs['property_id'])

print("\nNumber of property_ids in logs not in prop_size:", len(non_matching_in_logs))
print("\nSample of non-matching property_ids in logs (first 10):")
print(list(non_matching_in_logs)[:10])

# Perform a left join to see unmatched rows
merged_df = df_logs.merge(df_prop_size, left_on='property_id', right_on='id', how='left', indicator=True)
print("\nMerge result:")
print(merged_df['_merge'].value_counts())

Unique property_id in df_logs: 700
Unique ids in df_prop_size: 784

Number of common property IDs: 676

Percentage of property_id in logs that match prop_size: 96.57%
Percentage of ids in prop_size that match logs: 86.22%

Sample of matching property IDs (first 10):
['2c4d3880-b3eb-4ff2-8a24-52c28f0d239d', 'b3915d54-19cb-43ff-b33d-3d4b3edc76f5', '7d972ac4-62cf-4c6f-a824-de6a91a077f6', 'f3c7c831-4fc8-4867-9bfe-940ce4b41abd', 'cee9d389-0a1c-4ae3-b4b5-751b9ad13395', '56fdf5a0-6f6a-4166-bfdc-f4544a8f0ec0', '0b372bad-6c5f-4ff7-ab2f-5f9b380d5e6f', 'ef33ba45-3393-49f0-bef2-84c5d251dfe3', '23cdfb03-d755-4e6e-a492-cde3efc52aa8', '46237996-96a3-4be9-acbf-957ec4259019']

Number of property_ids in logs not in prop_size: 7208

Sample of non-matching property_ids in logs (first 10):
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

Merge result:
_merge
both          1768602
left_only       12505
right_only          0
Name: count, dtype: int64


In [8]:
# Check join logic
# Check join between df_prop.property_type and df_prop_type.id

print("Unique property_type in df_prop:", df_prop['property_type'].nunique())
print("Unique ids in df_prop_type:", df_prop_type['id'].nunique())

# Check overlap between keys
common_keys = set(df_prop['property_type']) & set(df_prop_type['id'])
print("\nNumber of common property types:", len(common_keys))

# Percentage of matching keys
prop_match_percentage = len(common_keys) / df_prop['property_type'].nunique() * 100
prop_type_match_percentage = len(common_keys) / df_prop_type['id'].nunique() * 100

print(f"\nPercentage of property_type in prop that match prop_type: {prop_match_percentage:.2f}%")
print(f"Percentage of ids in prop_type that match prop: {prop_type_match_percentage:.2f}%")

# Perform a left join to see unmatched rows
merged_df = df_prop.merge(df_prop_type, left_on='property_type', right_on='id', how='left', indicator=True)
print("\nMerge result:")
print(merged_df['_merge'].value_counts())

# Check unmatched property types
unmatched = merged_df[merged_df['_merge'] == 'left_only']
print("\nUnmatched property types:")
print(unmatched['property_type'].unique())

Unique property_type in df_prop: 13
Unique ids in df_prop_type: 20

Number of common property types: 13

Percentage of property_type in prop that match prop_type: 100.00%
Percentage of ids in prop_type that match prop: 65.00%

Merge result:
_merge
both          801
left_only       6
right_only      0
Name: count, dtype: int64

Unmatched property types:
[None]


In [3]:
# Decorate log data with property metadata

df_logs_exp = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_exp.pkl')

df_logs_metadata = df_logs_exp.copy()

# Left join to append property_type from df_prop
df_logs_metadata = df_logs_metadata.merge(
   df_prop[['id', 'property_type']],
   left_on='property_id',
   right_on='id',
   how='left'
)

# Drop the redundant id column
df_logs_metadata = df_logs_metadata.drop(columns=['id'])

# Left join to append name from df_prop_type
df_logs_metadata = df_logs_metadata.merge(
   df_prop_type[['id', 'name']],
   left_on='property_type',
   right_on='id',
   how='left'
)

# Drop the redundant id column
df_logs_metadata = df_logs_metadata.drop(columns=['id'])

# Left join to append Parking Space Count from df_prop_size
df_logs_metadata = df_logs_metadata.merge(
   df_prop_size[['id', 'Parking Space Count']],
   left_on='property_id',
   right_on='id',
   how='left'
)

# Save the data
df_logs_metadata.to_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_metadata.pkl')


NameError: name 'df_prop' is not defined

In [53]:
# Data clean-up

# Drop and re-name features
df_logs_metadata = df_logs_metadata.drop(columns=['id','property_type'])
df_logs_metadata = df_logs_metadata[['property_id','name','Parking Space Count','transaction_id','timestamp','user_id','unit','value']]
df_logs_metadata = df_logs_metadata.rename(columns={'name': 'property_type'})
df_logs_metadata = df_logs_metadata.rename(columns={'Parking Space Count': 'property_size'})


#Cast numeric formats
df_logs_metadata['property_size'] = pd.to_numeric(df_logs_metadata['property_size'], errors='coerce').fillna(0).astype(int)
df_logs_metadata['value'] = pd.to_numeric(df_logs_metadata['value'], errors='coerce').fillna(0).astype(int)

# Fix timestamp format and convert to Pacific
def is_valid_timestamp(timestamp):
    try:
        pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%S.%fZ')
        return True
    except:
        try:
            pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%SZ')
            return True
        except:
            try:
                pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%S.%f%z')
                return True
            except:
                return False

def remove_invalid_timestamps(df):
    return df[df['timestamp'].apply(is_valid_timestamp)]

def truncate_timestamp(df):
    df['timestamp'] = df['timestamp'].str[:16]
    return df

def convert_to_pacific_time(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_convert('US/Pacific')
    return df

def process_timestamps(df_logs_metadata):
    df_logs_metadata = remove_invalid_timestamps(df_logs_metadata)
    df_logs_metadata = truncate_timestamp(df_logs_metadata)
    df_logs_metadata = convert_to_pacific_time(df_logs_metadata)

    return df_logs_metadata

# Usage
df_logs_metadata = process_timestamps(df_logs_metadata)

In [54]:
# Fix timestamp formats and convert to Pacific
def is_valid_timestamp(timestamp):
    try:
        pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%S.%fZ')
        return True
    except:
        try:
            pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%SZ')
            return True
        except:
            try:
                pd.to_datetime(timestamp, format='%Y-%m-%dT%H:%M:%S.%f%z')
                return True
            except:
                return False

def remove_invalid_timestamps(df):
    return df[df['timestamp'].apply(is_valid_timestamp)]

def truncate_timestamp(df):
    df['timestamp'] = df['timestamp'].str[:16]
    return df

def convert_to_pacific_time(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_convert('US/Pacific')
    return df

def process_timestamps(df_logs_metadata):
    df_logs_metadata = remove_invalid_timestamps(df_logs_metadata)
    df_logs_metadata = truncate_timestamp(df_logs_metadata)
    df_logs_metadata = convert_to_pacific_time(df_logs_metadata)

    return df_logs_metadata

# Usage
df_logs_metadata = process_timestamps(df_logs_metadata)

## Data Exploration

In [56]:
df_logs_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3289867 entries, 0 to 3289866
Data columns (total 8 columns):
 #   Column          Dtype                     
---  ------          -----                     
 0   property_id     object                    
 1   property_type   object                    
 2   property_size   int64                     
 3   transaction_id  int64                     
 4   timestamp       datetime64[ns, US/Pacific]
 5   user_id         object                    
 6   unit            object                    
 7   value           int64                     
dtypes: datetime64[ns, US/Pacific](1), int64(3), object(4)
memory usage: 200.8+ MB


In [64]:
# Does each transaction_id have a unique timestamp:

import pandas as pd



# Create a new column with the combination of transaction_id and timestamp
df_logs_metadata['transaction_timestamp_pair'] = df_logs_metadata['transaction_id'].astype(str) + '_' + df_logs_metadata['timestamp'].astype(str)

# Count the number of occurrences of each pair
pair_counts = df_logs_metadata['transaction_timestamp_pair'].value_counts()

# Check if there are any duplicates
duplicates = pair_counts[pair_counts > 1]

if duplicates.empty:
    print("transaction_id and timestamp pairs are unique.")
    is_unique = True
else:
    print(f"Found {len(duplicates)} duplicate transaction_id and timestamp pairs.")
    print("Examples of duplicates:")
    print(duplicates.head())
    is_unique = False

print(f"\nTotal rows: {len(df_logs_metadata)}")
print(f"Unique pairs: {len(pair_counts)}")
print(f"Are all pairs unique? {is_unique}")

# If you want to see the actual duplicate rows:
if not is_unique:
    print("\nExample of duplicate rows:")
    duplicate_pairs = duplicates.index[:5]  # Get the first 5 duplicate pairs
    for pair in duplicate_pairs:
        print(df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'] == pair][['transaction_id', 'timestamp', 'property_id', 'user_id', 'value', 'unit']])
        print()

# Additional analysis on duplicates if they exist
if not is_unique:
    duplicate_df = df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'].isin(duplicates.index)]

    print("\nAnalysis of duplicate pairs:")
    print(f"Number of unique property_ids in duplicates: {duplicate_df['property_id'].nunique()}")
    print(f"Number of unique user_ids in duplicates: {duplicate_df['user_id'].nunique()}")
    print(f"Number of unique values in duplicates: {duplicate_df['value'].nunique()}")
    print(f"Number of unique units in duplicates: {duplicate_df['unit'].nunique()}")

    print("\nMost common property_types in duplicates:")
    print(duplicate_df['property_type'].value_counts().head())

Found 506191 duplicate transaction_id and timestamp pairs.
Examples of duplicates:
transaction_timestamp_pair
1938384688_2024-10-05 10:08:00-07:00    1540
464091764_2024-11-30 20:05:00-08:00     1440
1425252740_2024-11-23 16:18:00-08:00    1380
82110937_2024-12-24 13:24:00-08:00      1052
1388746211_2024-11-17 09:13:00-08:00     916
Name: count, dtype: int64

Total rows: 3289867
Unique pairs: 1766678
Are all pairs unique? False

Example of duplicate rows:
         transaction_id                 timestamp  \
1341517      1938384688 2024-10-05 10:08:00-07:00   
1341518      1938384688 2024-10-05 10:08:00-07:00   
1341519      1938384688 2024-10-05 10:08:00-07:00   
1341520      1938384688 2024-10-05 10:08:00-07:00   
1341620      1938384688 2024-10-05 10:08:00-07:00   
...                 ...                       ...   
1436751      1938384688 2024-10-05 10:08:00-07:00   
1436777      1938384688 2024-10-05 10:08:00-07:00   
1436778      1938384688 2024-10-05 10:08:00-07:00   
1436779   

In [65]:
# Is the combination of transaction_id, timestamp and unit unique ?
import pandas as pd

# Assuming df_logs_metadata is your existing DataFrame

# Create a new column with the combination of transaction_id, timestamp, and unit
df_logs_metadata['transaction_timestamp_unit_combo'] = (
    df_logs_metadata['transaction_id'].astype(str) + '_' +
    df_logs_metadata['timestamp'].astype(str) + '_' +
    df_logs_metadata['unit']
)

# Count the number of occurrences of each combination
combo_counts = df_logs_metadata['transaction_timestamp_unit_combo'].value_counts()

# Check if there are any duplicates
duplicates = combo_counts[combo_counts > 1]

if duplicates.empty:
    print("The combination of transaction_id, timestamp, and unit is unique.")
    is_unique = True
else:
    print(f"Found {len(duplicates)} duplicate combinations of transaction_id, timestamp, and unit.")
    print("Examples of duplicates:")
    print(duplicates.head())
    is_unique = False

print(f"\nTotal rows: {len(df_logs_metadata)}")
print(f"Unique combinations: {len(combo_counts)}")
print(f"Are all combinations unique? {is_unique}")

# If you want to see the actual duplicate rows:
if not is_unique:
    print("\nExample of duplicate rows:")
    duplicate_combos = duplicates.index[:5]  # Get the first 5 duplicate combinations
    for combo in duplicate_combos:
        print(df_logs_metadata[df_logs_metadata['transaction_timestamp_unit_combo'] == combo][['transaction_id', 'timestamp', 'unit', 'property_id', 'user_id', 'value']])
        print()

# Additional analysis on duplicates if they exist
if not is_unique:
    duplicate_df = df_logs_metadata[df_logs_metadata['transaction_timestamp_unit_combo'].isin(duplicates.index)]

    print("\nAnalysis of duplicate combinations:")
    print(f"Number of unique property_ids in duplicates: {duplicate_df['property_id'].nunique()}")
    print(f"Number of unique user_ids in duplicates: {duplicate_df['user_id'].nunique()}")
    print(f"Number of unique values in duplicates: {duplicate_df['value'].nunique()}")

    print("\nMost common property_types in duplicates:")
    print(duplicate_df['property_type'].value_counts().head())




# Example of a "duplicate" transaction_id: 1938384688
result = df_logs_metadata[df_logs_metadata['transaction_id'] == 1938384688]
print(result)






Found 995109 duplicate combinations of transaction_id, timestamp, and unit.
Examples of duplicates:
transaction_timestamp_unit_combo
1938384688_2024-10-05 10:08:00-07:00_A    770
1938384688_2024-10-05 10:08:00-07:00_W    770
464091764_2024-11-30 20:05:00-08:00_A     720
464091764_2024-11-30 20:05:00-08:00_W     720
1425252740_2024-11-23 16:18:00-08:00_A    690
Name: count, dtype: int64

Total rows: 3289867
Unique combinations: 2266379
Are all combinations unique? False

Example of duplicate rows:
         transaction_id                 timestamp unit  \
1341517      1938384688 2024-10-05 10:08:00-07:00    A   
1341518      1938384688 2024-10-05 10:08:00-07:00    A   
1341620      1938384688 2024-10-05 10:08:00-07:00    A   
1341621      1938384688 2024-10-05 10:08:00-07:00    A   
1341714      1938384688 2024-10-05 10:08:00-07:00    A   
...                 ...                       ...  ...   
1436724      1938384688 2024-10-05 10:08:00-07:00    A   
1436748      1938384688 2024-10-05

In [66]:
# Example of a "duplicate" transaction_id: 1938384688
result = df_logs_metadata[df_logs_metadata['transaction_id'] == 1938384688]
print(result)



                                  property_id property_type  property_size  \
1341517  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1341518  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1341519  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1341520  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1341620  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
...                                       ...           ...            ...   
1489393  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1490456  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1490457  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1490458  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   
1490459  51865921-029c-47fa-a3e8-2e8d3e45f26d  Condominiums            100   

         transaction_id                 timestamp  \
1341517   

In [68]:
# How often does transaction_id have multiple user_ids ?
# Only 1 example was found.

# Count unique user_ids for each transaction_id
user_id_counts = df_logs_metadata.groupby('transaction_id')['user_id'].nunique()

# Count unique property_ids for each transaction_id
property_id_counts = df_logs_metadata.groupby('transaction_id')['property_id'].nunique()

# Calculate results
total_transactions = len(df_logs_metadata['transaction_id'].unique())
transactions_with_multiple_users = (user_id_counts > 1).sum()
transactions_with_multiple_properties = (property_id_counts > 1).sum()

print(f"Total unique transactions: {total_transactions}")
print(f"Transactions with multiple user_ids: {transactions_with_multiple_users}")
print(f"Percentage of transactions with multiple user_ids: {transactions_with_multiple_users/total_transactions*100:.2f}%")
print(f"Transactions with multiple property_ids: {transactions_with_multiple_properties}")
print(f"Percentage of transactions with multiple property_ids: {transactions_with_multiple_properties/total_transactions*100:.2f}%")

# If you want to see examples of transactions with multiple user_ids or property_ids:
print("\nExamples of transactions with multiple user_ids:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(user_id_counts[user_id_counts > 1].index)].groupby('transaction_id')[['user_id', 'property_id']].head())

print("\nExamples of transactions with multiple property_ids:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(property_id_counts[property_id_counts > 1].index)].groupby('transaction_id')[['user_id', 'property_id']].head())

Total unique transactions: 158479
Transactions with multiple user_ids: 1
Percentage of transactions with multiple user_ids: 0.00%
Transactions with multiple property_ids: 1
Percentage of transactions with multiple property_ids: 0.00%

Examples of transactions with multiple user_ids:
                                      user_id  \
1356823  78a7ecf3-b926-4a18-b9fe-34a4c24be7dd   
1356852  78a7ecf3-b926-4a18-b9fe-34a4c24be7dd   
1362345  78a7ecf3-b926-4a18-b9fe-34a4c24be7dd   
1362805  78a7ecf3-b926-4a18-b9fe-34a4c24be7dd   
1363469  78a7ecf3-b926-4a18-b9fe-34a4c24be7dd   

                                  property_id  
1356823  21faa010-7a71-488a-a77c-30c463055104  
1356852  21faa010-7a71-488a-a77c-30c463055104  
1362345  21faa010-7a71-488a-a77c-30c463055104  
1362805  21faa010-7a71-488a-a77c-30c463055104  
1363469  21faa010-7a71-488a-a77c-30c463055104  

Examples of transactions with multiple property_ids:
                                      user_id  \
1356823  78a7ecf3-b926-4a18-b9

In [71]:
# How often does transaction_id have multiple year or month values ?
# Looks like its concentrated at the begining or ending of a month

# Extract year and month from the timestamp
df_logs_metadata['year'] = df_logs_metadata['timestamp'].dt.year
df_logs_metadata['month'] = df_logs_metadata['timestamp'].dt.month

# Count unique years for each transaction_id
year_counts = df_logs_metadata.groupby('transaction_id')['year'].nunique()

# Count unique months for each transaction_id
month_counts = df_logs_metadata.groupby('transaction_id')['month'].nunique()

# Calculate results
total_transactions = len(df_logs_metadata['transaction_id'].unique())
transactions_with_multiple_years = (year_counts > 1).sum()
transactions_with_multiple_months = (month_counts > 1).sum()

print(f"Total unique transactions: {total_transactions}")
print(f"Transactions spanning multiple years: {transactions_with_multiple_years}")
print(f"Percentage of transactions spanning multiple years: {transactions_with_multiple_years/total_transactions*100:.2f}%")
print(f"Transactions spanning multiple months: {transactions_with_multiple_months}")
print(f"Percentage of transactions spanning multiple months: {transactions_with_multiple_months/total_transactions*100:.2f}%")

# If you want to see examples of transactions spanning multiple years or months:
print("\nExamples of transactions spanning multiple years:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(year_counts[year_counts > 1].index)].groupby('transaction_id')[['timestamp', 'year', 'month']].head())

print("\nExamples of transactions spanning multiple months:")
print(df_logs_metadata[df_logs_metadata['transaction_id'].isin(month_counts[month_counts > 1].index)].groupby('transaction_id')[['timestamp', 'year', 'month']].head())

Total unique transactions: 158479
Transactions spanning multiple years: 0
Percentage of transactions spanning multiple years: 0.00%
Transactions spanning multiple months: 1985
Percentage of transactions spanning multiple months: 1.25%

Examples of transactions spanning multiple years:
Empty DataFrame
Columns: [timestamp, year, month]
Index: []

Examples of transactions spanning multiple months:
                        timestamp  year  month
2       2024-09-30 02:38:00-07:00  2024      9
3       2024-09-30 02:38:00-07:00  2024      9
4       2024-09-30 02:38:00-07:00  2024      9
5       2024-09-30 02:38:00-07:00  2024      9
16      2024-09-30 02:37:00-07:00  2024      9
...                           ...   ...    ...
3264709 2024-12-01 03:18:00-08:00  2024     12
3264848 2024-12-01 03:12:00-08:00  2024     12
3265402 2024-12-01 02:45:00-08:00  2024     12
3265652 2024-12-01 11:26:00-08:00  2024     12
3267425 2024-12-01 09:48:00-08:00  2024     12

[9906 rows x 3 columns]


In [1]:
# How often does transaction_id have multiple A or W non-0 values  ?

# Filter for non-zero 'A' and 'W' values
df_non_zero = df_logs_metadata[(df_logs_metadata['value'] != 0) &
                               (df_logs_metadata['unit'].isin(['A', 'W']))]

# Group by transaction_id and unit, then count non-zero values
value_counts = df_non_zero.groupby(['transaction_id', 'unit'])['value'].count().unstack(fill_value=0)

# Count transactions with multiple non-zero 'A' values
multiple_A = (value_counts['A'] > 1).sum()

# Count transactions with multiple non-zero 'W' values
multiple_W = (value_counts['W'] > 1).sum()

# Total unique transactions
total_transactions = df_logs_metadata['transaction_id'].nunique()

print(f"Total unique transactions: {total_transactions}")
print(f"Transactions with multiple non-zero 'A' values: {multiple_A}")
print(f"Percentage of transactions with multiple non-zero 'A' values: {multiple_A/total_transactions*100:.2f}%")
print(f"Transactions with multiple non-zero 'W' values: {multiple_W}")
print(f"Percentage of transactions with multiple non-zero 'W' values: {multiple_W/total_transactions*100:.2f}%")

# Examples of transactions with multiple non-zero 'A' or 'W' values
print("\nExamples of transactions with multiple non-zero 'A' values:")
print(df_non_zero[df_non_zero['transaction_id'].isin(value_counts[value_counts['A'] > 1].index) &
                  (df_non_zero['unit'] == 'A')].groupby('transaction_id').head())

print("\nExamples of transactions with multiple non-zero 'W' values:")
print(df_non_zero[df_non_zero['transaction_id'].isin(value_counts[value_counts['W'] > 1].index) &
                  (df_non_zero['unit'] == 'W')].groupby('transaction_id').head())




NameError: name 'df_logs_metadata' is not defined

In [63]:
# Check uniqueness of transaction_id across different column combinations
def check_uniqueness(df, columns):
    unique_combinations = df.groupby(columns)['transaction_id'].nunique()
    print(f"\nUniqueness check for {columns}:")
    print("Number of unique transaction_ids per group:", unique_combinations.max())

    if unique_combinations.max() > 1:
        print("\nGroups with multiple transaction_ids:")
        print(unique_combinations[unique_combinations > 1])

# Check various column combinations
check_columns = [
    ['user_id', 'timestamp', 'property_id', 'value', 'unit'],
    ['user_id', 'property_id', 'value', 'unit'],
    ['timestamp', 'property_id', 'value', 'unit'],
    ['user_id', 'timestamp', 'value', 'unit']
]

for cols in check_columns:
    check_uniqueness(df_logs_metadata, cols)

# Additional overall statistics
print("\nOverall unique counts:")
print("Total rows:", len(df_logs_metadata))
print("Unique transaction_ids:", df_logs_metadata['transaction_id'].nunique())
print("Unique combinations:",
    df_logs_metadata.groupby(['user_id', 'timestamp', 'property_id', 'value', 'unit'])['transaction_id'].nunique().max())


Uniqueness check for ['user_id', 'timestamp', 'property_id', 'value', 'unit']:
Number of unique transaction_ids per group: 2

Groups with multiple transaction_ids:
user_id                               timestamp                  property_id                           value  unit
18bf9385-29e3-4d8b-8b5b-406e80ed0d5a  2024-09-07 09:44:00-07:00  ab1035ee-0f5e-445a-bc53-b40c3cc5d8ff  0      A       2
4268c527-5452-4c7c-b10d-25373bafc1e3  2024-09-09 20:59:00-07:00  ac76db18-231d-4dad-928c-c42fcb9ae2f9  0      A       2
                                      2024-10-28 09:18:00-07:00  348c41a5-baaa-4275-b8c5-4d9e6ba8f97d  10     A       2
                                      2024-11-27 10:34:00-08:00  795cc566-8d58-4c54-b9ea-501671bb2b45  0      A       2
5378280e-d6d3-4c22-8304-37ee3993ca7e  2024-10-15 12:20:00-07:00  9af54d24-5521-46d4-8db8-6d42227efb04  30     A       2
8fd904c0-b577-4fd5-9409-5e35a6440337  2024-09-21 23:39:00-07:00  53e9cab2-aed2-4c35-ab39-e8375825e6fb  0      A       2


In [ ]:
import pandas as pd

# Assuming df_logs_metadata is your existing DataFrame

# Create a new column with the combination of transaction_id and timestamp
df_logs_metadata['transaction_timestamp_pair'] = df_logs_metadata['transaction_id'].astype(str) + '_' + df_logs_metadata['timestamp'].astype(str)

# Count the number of occurrences of each pair
pair_counts = df_logs_metadata['transaction_timestamp_pair'].value_counts()

# Check if there are any duplicates
duplicates = pair_counts[pair_counts > 1]

if duplicates.empty:
    print("transaction_id and timestamp pairs are unique.")
    is_unique = True
else:
    print(f"Found {len(duplicates)} duplicate transaction_id and timestamp pairs.")
    print("Examples of duplicates:")
    print(duplicates.head())
    is_unique = False

print(f"\nTotal rows: {len(df_logs_metadata)}")
print(f"Unique pairs: {len(pair_counts)}")
print(f"Are all pairs unique? {is_unique}")

# If you want to see the actual duplicate rows:
if not is_unique:
    print("\nExample of duplicate rows:")
    duplicate_pairs = duplicates.index[:5]  # Get the first 5 duplicate pairs
    for pair in duplicate_pairs:
        print(df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'] == pair][['transaction_id', 'timestamp', 'property_id', 'user_id', 'value', 'unit']])
        print()

# Additional analysis on duplicates if they exist
if not is_unique:
    duplicate_df = df_logs_metadata[df_logs_metadata['transaction_timestamp_pair'].isin(duplicates.index)]

    print("\nAnalysis of duplicate pairs:")
    print(f"Number of unique property_ids in duplicates: {duplicate_df['property_id'].nunique()}")
    print(f"Number of unique user_ids in duplicates: {duplicate_df['user_id'].nunique()}")
    print(f"Number of unique values in duplicates: {duplicate_df['value'].nunique()}")
    print(f"Number of unique units in duplicates: {duplicate_df['unit'].nunique()}")

    print("\nMost common property_types in duplicates:")
    print(duplicate_df['property_type'].value_counts().head())import matplotlib.pyplot as plt
import numpy as np

# Histogram of timestamps per transaction_id
timestamps_per_transaction = df_logs_enriched.groupby('transaction_id')['timestamp'].nunique()

plt.figure(figsize=(12, 5))

# Left plot - log scale
plt.subplot(1, 2, 1)
plt.hist(timestamps_per_transaction, bins=np.logspace(np.log10(1), np.log10(timestamps_per_transaction.max()), 50))
plt.xscale('log')
plt.yscale('log')
plt.title('Unique Timestamps per Transaction ID (Log Scale)')
plt.xlabel('Number of Unique Timestamps')
plt.ylabel('Count of Transaction IDs')
plt.grid(True)

# Right plot - properties per transaction
properties_per_transaction = df_logs_enriched.groupby('transaction_id')['property_id'].nunique()
plt.subplot(1, 2, 2)
plt.hist(properties_per_transaction, bins=50)
plt.title('Unique Properties per Transaction ID')
plt.xlabel('Number of Unique Properties')
plt.ylabel('Count of Transaction IDs')

plt.tight_layout()
plt.show()

print("\nTimestamps per transaction summary:")
print(timestamps_per_transaction.describe())
print("\nProperties per transaction summary:")
print(properties_per_transaction.describe())

# Print additional context
print("\nTotal number of unique transactions:", len(timestamps_per_transaction))
print("Total number of unique properties:", df_logs_enriched['property_id'].nunique())
print("Total number of timestamps:", df_logs_enriched['timestamp'].nunique())

## Engineer Features

In [ ]:
import pandas as pd
import pytz
import numpy as np
from datetime import datetime

def convert_to_pst_components(df):
    """Convert timestamp to PST and extract components."""
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601')
    df['timestamp'] = df['timestamp'].dt.tz_convert('US/Pacific')

    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute

    return df

def add_day_info(df):
    """Add day of week and weekend indicator."""
    df['day_of_week'] = df['timestamp'].dt.dayofweek + 1
    df['day_weekend'] = (df['day_of_week'] >= 6).astype(int)
    return df

def get_nearest_holiday(df):
    """Calculate proximity to major US holidays."""
    major_holidays = {
        '2024-01-01': "New Year's Day",
        '2024-01-15': "Martin Luther King Jr. Day",
        '2024-02-19': "Presidents Day",
        '2024-05-27': "Memorial Day",
        '2024-07-04': "Independence Day",
        '2024-09-02': "Labor Day",
        '2024-11-28': "Thanksgiving",
        '2024-12-25': "Christmas",
        '2025-01-01': "New Year's Day",
        '2025-01-20': "Martin Luther King Jr. Day",
        '2025-02-17': "Presidents Day",
        '2025-05-26': "Memorial Day",
        '2025-07-04': "Independence Day",
        '2025-09-01': "Labor Day",
        '2025-11-27': "Thanksgiving",
        '2025-12-25': "Christmas"
    }

    holiday_dates = pd.to_datetime(list(major_holidays.keys())).sort_values()
    holiday_dates_array = holiday_dates.values
    dates_array = pd.to_datetime(df['timestamp'].dt.date.unique()).values
    holiday_lookup = {}

    for date in dates_array:
        days_diff = np.abs((holiday_dates_array - date).astype('timedelta64[D]').astype(int))
        closest_idx = np.argmin(days_diff)
        closest_date = holiday_dates[closest_idx]

        holiday_lookup[pd.Timestamp(date).date()] = {
            'days_to_nearest_holiday': days_diff[closest_idx],
            'nearest_holiday_date': closest_date,
            'nearest_holiday_name': major_holidays[closest_date.strftime('%Y-%m-%d')]
        }

    # Create and merge holiday information
    df['date'] = df['timestamp'].dt.date
    result = pd.DataFrame.from_dict(holiday_lookup, orient='index')
    result.index = pd.to_datetime(result.index).date
    df = df.merge(result, left_on='date', right_index=True)
    df = df.drop('date', axis=1)

    return df

def process_timestamps(input_path, output_path):
    """Main function to process all timestamp-related features."""
    df = pd.read_pickle(input_path)
    df = convert_to_pst_components(df)
    df = add_day_info(df)
    df = get_nearest_holiday(df)
    df.to_pickle(output_path)
    return df

if __name__ == "__main__":
    input_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched.pkl'
    output_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_holidays.pkl'

    df_processed = process_timestamps(input_path, output_path)
    print(f"Processing complete. File saved to: {output_path}")

In [ ]:

# Add user and usage data
import pandas as pd

def add_unique_user_counts(df, group_cols, user_col):
    """Count unique users per group."""
    user_counts = df.groupby(group_cols)[user_col].nunique().reset_index()
    user_counts.rename(columns={user_col: 'unique_user_count'}, inplace=True)
    df = df.merge(user_counts, on=group_cols, how='left')
    return df

def add_usage_sums(df, group_cols):
    """Add sums of values for each unit type by group."""
    # Calculate sums for each unit type
    sums = df.groupby([*group_cols, 'unit_encoded'])['value'].sum().reset_index()

    # Pivot to create separate columns for A and W
    sums = sums.pivot(
        index=group_cols,
        columns='unit_encoded',
        values='value'
    ).reset_index()

    # Rename columns
    sums.rename(
        columns={
            0: 'sum_value_A',   # Amps were encoded as 0
            1: 'sum_value_Wh'   # Watts were encoded as 1
        },
        inplace=True
    )

    return df.merge(sums, on=group_cols, how='left')

if __name__ == "__main__":
    input_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_holidays.pkl'
    output_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl'

    # Read data
    df = pd.read_pickle(input_path)

    # Extract datetime components
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour

    # Define grouping columns after datetime components are created
    group_cols = ['property_id', 'year', 'month', 'day', 'hour']

    # Add user and usage metrics
    df = add_unique_user_counts(df, group_cols, 'user_id')
    df = add_usage_sums(df, group_cols)

    # Save results
    df.to_pickle(output_path)
    print(f"Processing complete. File saved to: {output_path}")

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

In [ ]:
df.info()

In [ ]:
# Halt
sys.exit()

## Check for colinearity

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert columns to numeric, handling errors
columns_to_analyze = [
    "day_of_week",
    "days_to_nearest_holiday",
    "month",
    "day",
    "hour",
    "unique_user_count",
    "sum_value_A",
    "sum_value_Wh"
]

# Create a clean dataframe for analysis
df_clean = df[columns_to_analyze].copy()

# Convert each column to numeric, handling errors
for col in columns_to_analyze:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # Print info about conversion
    print(f"\nColumn: {col}")
    print(f"Null values after conversion: {df_clean[col].isnull().sum()}")
    print(f"Sample unique values: {df_clean[col].dropna().sample(5).tolist()}")

# Calculate correlations for cleaned numeric columns
correlations = df_clean.corr()

# Create correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

# Basic summary statistics
print("\nSummary Statistics:")
print(df_clean.describe())

# Check for patterns across categorical variables
print("\nMean energy consumption (sum_value_Wh) by:")
print("\nDay of Week:")
print(df_clean.groupby('day_of_week')['sum_value_Wh'].mean().sort_values(ascending=False))

print("\nHour of Day:")
print(df_clean.groupby('hour')['sum_value_Wh'].mean().sort_values(ascending=False))

print("\nMonth:")
print(df_clean.groupby('month')['sum_value_Wh'].mean().sort_values(ascending=False))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load and clean data
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert columns to numeric
df['hour'] = pd.to_numeric(df['hour'], errors='coerce')
df['sum_value_Wh'] = pd.to_numeric(df['sum_value_Wh'], errors='coerce')
df['unique_user_count'] = pd.to_numeric(df['unique_user_count'], errors='coerce')

# Group by hour and calculate various statistics
hourly_stats = df.groupby('hour').agg({
    'sum_value_Wh': ['mean', 'median', 'std', 'count'],
    'unique_user_count': 'mean'
}).round(2)

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Energy consumption pattern
ax1.plot(hourly_stats[('sum_value_Wh', 'mean')], marker='o')
ax1.fill_between(range(24),
                 hourly_stats[('sum_value_Wh', 'mean')] - hourly_stats[('sum_value_Wh', 'std')],
                 hourly_stats[('sum_value_Wh', 'mean')] + hourly_stats[('sum_value_Wh', 'std')],
                 alpha=0.2)
ax1.set_title('24-Hour Energy Consumption Pattern')
ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Average Energy Consumption (Wh)')
ax1.grid(True)

# Plot 2: Users vs Energy
ax2.scatter(hourly_stats[('unique_user_count', 'mean')],
           hourly_stats[('sum_value_Wh', 'mean')],
           alpha=0.6)
# Add hour labels to each point
for i in range(24):
    ax2.annotate(str(i),
                (hourly_stats[('unique_user_count', 'mean')][i],
                 hourly_stats[('sum_value_Wh', 'mean')][i]))
ax2.set_title('Users vs Energy Consumption by Hour')
ax2.set_xlabel('Average Number of Users')
ax2.set_ylabel('Average Energy Consumption (Wh)')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Print detailed statistics for peak hours
peak_hours = hourly_stats.nlargest(5, ('sum_value_Wh', 'mean'))
print("\nPeak Hours Analysis:")
print(peak_hours)

# Calculate relative increase during peak vs off-peak
off_peak_mean = hourly_stats.loc[0:6, ('sum_value_Wh', 'mean')].mean()
peak_mean = hourly_stats.loc[16:19, ('sum_value_Wh', 'mean')].mean()
increase_factor = peak_mean / off_peak_mean

print(f"\nPeak vs Off-peak Analysis:")
print(f"Average off-peak consumption (midnight-6am): {off_peak_mean:.2f} Wh")
print(f"Average peak consumption (4pm-7pm): {peak_mean:.2f} Wh")
print(f"Peak hours consume {increase_factor:.1f}x more energy than off-peak hours")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert sum_value_Wh to numeric
df['sum_value_Wh'] = pd.to_numeric(df['sum_value_Wh'], errors='coerce')

# Create a single property type column for easier plotting
prop_type_cols = [col for col in df.columns if col.startswith('prop_type_')]
df['property_type'] = np.nan

for col in prop_type_cols:
    mask = df[col] == 1
    df.loc[mask, 'property_type'] = col.replace('prop_type_', '')

# Create box plot
plt.figure(figsize=(15, 8))
sns.boxplot(x='property_type', y='sum_value_Wh', data=df)
plt.xticks(rotation=45, ha='right')
plt.title('Energy Consumption Distribution by Property Type')
plt.xlabel('Property Type')
plt.ylabel('Energy Consumption (Wh)')
plt.tight_layout()
plt.show()

# Print basic stats for each property type
print("\nBasic statistics by property type:")
stats = df.groupby('property_type')['sum_value_Wh'].describe()
print(stats)

# Count number of observations for each property type
print("\nNumber of observations per property type:")
counts = df['property_type'].value_counts()
print(counts)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Convert to numeric
df['sum_value_Wh'] = pd.to_numeric(df['sum_value_Wh'], errors='coerce')
df['unique_user_count'] = pd.to_numeric(df['unique_user_count'], errors='coerce')

# Create a single property type column
prop_type_cols = [col for col in df.columns if col.startswith('prop_type_')]
df['property_type'] = np.nan

for col in prop_type_cols:
    mask = df[col] == 1
    df.loc[mask, 'property_type'] = col.replace('prop_type_', '')

# Calculate energy per user
df['energy_per_user'] = df['sum_value_Wh'] / df['unique_user_count']

# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Plot 1: Total Energy
sns.boxplot(x='property_type', y='sum_value_Wh', data=df, ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.set_title('Total Energy Consumption by Property Type')
ax1.set_xlabel('Property Type')
ax1.set_ylabel('Energy Consumption (Wh)')

# Plot 2: Energy per User
sns.boxplot(x='property_type', y='energy_per_user', data=df, ax=ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.set_title('Energy Consumption per User by Property Type')
ax2.set_xlabel('Property Type')
ax2.set_ylabel('Energy Consumption per User (Wh/user)')

plt.tight_layout()
plt.show()

# Print statistics
print("\nStatistics by property type:")
stats = df.groupby('property_type').agg({
    'sum_value_Wh': ['count', 'mean'],
    'unique_user_count': 'mean',
    'energy_per_user': 'mean'
}).round(2)

print(stats)

In [ ]:
# First, let's look at the raw data before any processing
print("Original data counts by property type:")
for col in prop_type_cols:
    print(f"\n{col}:")
    print(f"Number of 1s: {df[col].sum()}")
    print(f"Number of non-null values: {df[col].count()}")

# Let's also check for nulls in key columns
print("\nNull values in key columns:")
print(df[['sum_value_Wh', 'unique_user_count']].isnull().sum())

# Let's look at the data before any type conversion
print("\nSample of raw data before conversion:")
sample_data = df[['property_type', 'sum_value_Wh', 'unique_user_count']].head(10)
print(sample_data)

# Check data types of key columns
print("\nData types of columns:")
print(df.dtypes)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load your dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

def clean_numeric_string(value):
    """Clean numeric string by taking the first valid number."""
    if pd.isna(value):
        return 0
    # Convert to string if not already
    value = str(value)
    # Find first number (integer or decimal)
    parts = value.split('.')
    if not parts:
        return 0
    try:
        # Take first valid number
        return float(parts[0])
    except ValueError:
        return 0

# Clean and convert sum_value columns
df['sum_value_A'] = df['sum_value_A'].apply(clean_numeric_string).astype(int)
df['sum_value_Wh'] = df['sum_value_Wh'].apply(clean_numeric_string).astype(int)
df['unique_user_count'] = df['unique_user_count'].astype(int)

# Select only numerical features for VIF calculation
numerical_columns = [
    "day_of_week",
    "day_weekend",
    "days_to_nearest_holiday",
    "year",
    "month",
    "day",
    "hour",
    "unique_user_count",
    "sum_value_A",
    "sum_value_Wh"
]

# Prepare the DataFrame for VIF calculation
X = df[numerical_columns].copy()

# Check for NaN and inf values
print(f"NaN values before VIF calculation: {X.isna().sum().sum()}")
print(f"Inf values before VIF calculation: {((X == np.inf) | (X == -np.inf)).sum().sum()}")

# Handle NaN and inf values
X = X.fillna(0)  # Replace NaN values with 0 or other strategy
X.replace([np.inf, -np.inf], 0, inplace=True)

# Check again after handling NaN and inf values
print(f"NaN values after VIF calculation: {X.isna().sum().sum()}")
print(f"Inf values after VIF calculation: {((X == np.inf) | (X == -np.inf)).sum().sum()}")

# Add a constant column for intercept
X['intercept'] = 1

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Drop the constant column after VIF calculation
vif_data = vif_data[vif_data["Feature"] != "intercept"]

# Display the VIF values
print(vif_data)

In [ ]:
# Drop suspect features and run VIF again

# Load the dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Drop suspect features
df = df.drop(['day_weekend', 'year'], axis=1)

# Define numerical columns
numerical_columns = [
    "day_of_week",
    "days_to_nearest_holiday",
    "month",
    "day",
    "hour",
    "unique_user_count",
    "sum_value_A",
    "sum_value_Wh"
]

# Convert columns to numeric type and handle any non-numeric values
X = df[numerical_columns].copy()
for column in numerical_columns:
    # Convert to numeric, coerce any non-numeric values to NaN
    X[column] = pd.to_numeric(X[column], errors='coerce')

    # Fill NaN values with the median of the column
    X[column] = X[column].fillna(X[column].median())

# Add a constant column for intercept
X['intercept'] = 1

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Drop the constant column after VIF calculation
vif_data = vif_data[vif_data["Feature"] != "intercept"]

# Sort VIF values in descending order
vif_data = vif_data.sort_values('VIF', ascending=False)

# Display the VIF values
print("\nVariance Inflation Factors:")
print(vif_data)

# Prep df for regression/ANOVA

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

# Load the dataset
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/df_logs_enriched_with_usage.pkl')

# Define the dependent variable (y) - in this case let's use sum_value_Wh as our target
y = "sum_value_Wh"

# Define independent variables (features to compare groups)
independent_vars = [
    "day_of_week",
    "days_to_nearest_holiday",
    "month",
    "day",
    "hour",
    "unique_user_count"
    #"sum_value_A"
]

# Convert columns to numeric type and handle any non-numeric values
X = df[independent_vars].copy()
for column in independent_vars:
    X[column] = pd.to_numeric(X[column], errors='coerce')
    X[column] = X[column].fillna(X[column].median())

# Convert y to numeric
df[y] = pd.to_numeric(df[y], errors='coerce')
df[y] = df[y].fillna(df[y].median())

# Function to run one-way ANOVA for each independent variable against y
def run_anova_with_target(df, independent_vars, y):
    results = []

    for var in independent_vars:
        # Create groups based on the independent variable
        groups = []
        # Create 5 groups using quantiles for continuous variables
        df['group'] = pd.qcut(df[var], q=5, labels=['G1', 'G2', 'G3', 'G4', 'G5'])

        # Get the y values for each group
        for group in df['group'].unique():
            groups.append(df[df['group'] == group][y].values)

        # Perform one-way ANOVA
        f_stat, p_val = stats.f_oneway(*groups)

        results.append({
            'Independent Variable': var,
            'F-statistic': f_stat,
            'p-value': p_val
        })

    return pd.DataFrame(results)

# Run ANOVA
anova_results = run_anova_with_target(df, independent_vars, y)

# Sort results by p-value
anova_results_sorted = anova_results.sort_values('p-value')

# Display results
pd.set_option('display.float_format', lambda x: '{:.10f}'.format(x) if x < 0.0001 else '{:.4f}'.format(x))
print(f"\nOne-way ANOVA Results (dependent variable: {y}):")
print(anova_results_sorted)

# Add significance indicators
anova_results_sorted['Significance'] = ['***' if p < 0.001
                                      else '**' if p < 0.01
                                      else '*' if p < 0.05
                                      else 'ns' for p in anova_results_sorted['p-value']]

print("\nSignificance levels:")
print("***: p < 0.001")
print("**: p < 0.01")
print("*: p < 0.05")
print("ns: not significant")

In [ ]:
import numpy as np
# Fit a regression model
X = sm.add_constant(df_sampled['unique_user_count'])  # Add intercept
model = sm.OLS(df_sampled['hour_sum_value_A'], X).fit()

# Predict values for regression line
predictions = model.predict(X)

# Plot scatter with regression line
plt.figure(figsize=(8, 6))
plt.scatter(df_sampled['unique_user_count'], df_sampled['hour_sum_value_A'], alpha=0.6, label='Data Points')
plt.plot(df_sampled['unique_user_count'], predictions, color='red', label='Regression Line')
plt.title('Regression Plot: Unique User Count vs Hour Sum Value 0')
plt.xlabel('Unique User Count')
plt.ylabel('Hour hour_sum_value_A')
plt.legend()
plt.show()


# Ensure property_id is treated as a categorical variable
df_sampled['property_id'] = df_sampled['property_id'].astype('category')

# Prepare the formula for ANOVA
independent_vars = ['unique_user_count', 'property_id'] + [col for col in df_sampled.columns if col.startswith('day_')]
formula = 'hour_sum_value_A ~ ' + ' + '.join(independent_vars)

# Fit the model
model = ols(formula, data=df_sampled).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA results
print(anova_results)


In [ ]:
# Calculate the IQR for the column with potential outliers
Q1 = df_sampled['hour_sum_value_A'].quantile(0.25)
Q3 = df_sampled['hour_sum_value_A'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df_filtered = df_sampled[(df_sampled['hour_sum_value_A'] >= lower_bound) & (df_sampled['hour_sum_value_A'] <= upper_bound)]


In [ ]:
## Create a property lookup

import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Function to fetch non-transposed data
def fetch_non_transposed_data(cursor, table):
    query = f"SELECT * FROM {table};"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=column_names)
    return df

# List of tables to process
tables = [
    "location",
    "properties"
]

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Fetch non-transposed data
        df_non_transposed = fetch_non_transposed_data(cursor, table)

        # Write the DataFrame to CSV with new naming convention
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/{table}_table_extract.csv'
        df_non_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")



In [ ]:
import numpy as np
# Fit a regression model
X = sm.add_constant(df_sampled['unique_user_count'])  # Add intercept
model = sm.OLS(df_sampled['hour_sum_value_A'], X).fit()

# Predict values for regression line
predictions = model.predict(X)

# Plot scatter with regression line
plt.figure(figsize=(8, 6))
plt.scatter(df_filtered['unique_user_count'], df_filtered['hour_sum_value_A'], alpha=0.6, label='Data Points')
plt.plot(df_filtered['unique_user_count'], predictions, color='red', label='Regression Line')
plt.title('Regression Plot: Unique User Count vs Hour Sum Value 0')
plt.xlabel('Unique User Count')
plt.ylabel('Hour hour_sum_value_A')
plt.legend()
plt.show()


# Ensure property_id is treated as a categorical variable
df_filtered['property_id'] = df_filtered['property_id'].astype('category')

# Prepare the formula for ANOVA
independent_vars = ['unique_user_count', 'property_id'] + [col for col in df_filtered.columns if col.startswith('day_')]
formula = 'hour_sum_value_A ~ ' + ' + '.join(independent_vars)

# Fit the model
model = ols(formula, data=df_filtered).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA results
print(anova_results)

In [ ]:
# Decorate data with engineered values

from datetime import datetime
import pytz

# Function to convert to PST and extract datetime
def convert_to_pst_as_datetime(timestamp):
    # Parse the UTC timestamp
    utc_time = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Set timezone to UTC
    utc_time = utc_time.replace(tzinfo=pytz.UTC)
    # Convert to PST
    pst_time = utc_time.astimezone(pytz.timezone('US/Pacific'))
    # Truncate to day, month, year, and hour (zero minutes and seconds)
    return pst_time.replace(minute=0, second=0, microsecond=0)

# Apply the function to convert timestamp
df_a_s_o['time_sample'] = df_a_s_o['timestamp'].apply(convert_to_pst_as_datetime)

# Add a column for day of the week (0 = Monday, 6 = Sunday)
df_a_s_o['day_of_week'] = df_a_s_o['time_sample'].dt.dayofweek

# Add a column for hour of the day (24hr format)
df_a_s_o['hour_of_day'] = df_a_s_o['time_sample'].dt.hour

# Add a column for ISO week number
df_a_s_o['week_number'] = df_a_s_o['time_sample'].dt.isocalendar().week

# Add in count of unique users
df_a_s_o['unique_user_count'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['user_id']
    .transform('nunique')
)

# Add in sum of unit_a
df_a_s_o['sum_of_unit_a'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['unit_a']
    .transform('sum')
)

# Add in sum of watt_h
df_a_s_o['sum_of_unit_wh'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['unit_wh']
    .transform('sum')
)

# Print the updated DataFrame
print(df_a_s_o)

In [ ]:
# Data Check
print(df_a_s_o['week_number'].unique())


# Calculate the overall count of unique user IDs
unique_user_count = df_a_s_o['user_id'].nunique()

# Calculate the sum of unit_a
sum_of_unit_a = df_a_s_o['unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = df_a_s_o['unit_wh'].sum()

# Print the results
print(f"Unique User Count: {unique_user_count}")
print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78

In [ ]:
df_a_s_o.info()

In [ ]:
# Reduce the DataFrame to unique rows based on the specified columns
reduced_df = df_a_s_o.drop_duplicates(
    subset=['day_of_week', 'hour_of_day', 'week_number', 'unique_user_count', 'sum_of_unit_a', 'sum_of_unit_wh']
)

# Keep only the specified columns
reduced_df = reduced_df[['day_of_week', 'hour_of_day', 'week_number', 'unique_user_count', 'sum_of_unit_a', 'sum_of_unit_wh']]

# Display the resulting DataFrame
print(reduced_df.info())
print(reduced_df.head())


In [ ]:

# Calculate the sum of unit_a
sum_of_unit_a = reduced_df['sum_of_unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = reduced_df['sum_of_unit_wh'].sum()

# Print the results

print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78



In [ ]:
# Write a local file to take a look

df_a_s_o.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/df_a_s_o.csv', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.regplot(x='unique_user_count', y='sum_of_unit_wh', data=df_a_s_o, scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'})
plt.xlabel('User unique_user_count Count')
plt.ylabel('Total Unit WH')
plt.title('Regression Plot: User ID Count vs. Total Unit WH')
plt.show()

In [ ]:
df_a_s_o.info()

In [ ]:
# Data is week 32 through week 44 (12)
# So below, there is no week lag1 value for week 32 because it is the first

# Identify the peak total_unit_wh for each week
peak_weekly_data = df.loc[df.groupby('week_number')['sum_of_unit_wh'].idxmax()]

# Sort by week number to ensure correct lagging
peak_weekly_data = peak_weekly_data.sort_values('week_number')

# Add only lag_1 features
peak_weekly_data['lag_1_day_of_week'] = peak_weekly_data['day_of_week'].shift(1)
peak_weekly_data['lag_1_hour'] = peak_weekly_data['hour_of_day'].shift(1)

# Drop rows with insufficient lag (week 1)
peak_weekly_data = peak_weekly_data.dropna()

# Retain only relevant columns
peak_weekly_data = peak_weekly_data[['week_number', 'day_of_week', 'hour_of_day', 'lag_1_day_of_week', 'lag_1_hour']]

print("Updated DataFrame:")
print(peak_weekly_data)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Features (lagged day of week and hour) and target (day of week)
X = peak_weekly_data[['lag_1_day_of_week', 'lag_1_hour']]
y = peak_weekly_data['day_of_week']  # Target: Day of the week

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Day of Week Prediction Accuracy:", accuracy)

# Display true vs predicted values
results = pd.DataFrame({'True Day': y_test, 'Predicted Day': y_pred})
print("\nTrue vs Predicted Days of the Week:")
print(results)


In [ ]:


# Feature importance for day_of_week classification
clf_importances = clf.feature_importances_
plt.barh(X.columns, clf_importances)
plt.title("Feature Importance for Day of Week Prediction")
plt.show()

# Feature importance for hour regression
reg_importances = reg.feature_importances_
plt.barh(X.columns, reg_importances)
plt.title("Feature Importance for Hour Prediction")
plt.show()


### Data Exploration

## Appendix

####AWS Tables

In [ ]:

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Execute a query to fetch all table names
    query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """

    cursor.execute(query)
    tables = cursor.fetchall()

    # Print the table names
    for table in tables:
        print(table[0])

except Exception as error:
    print(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        print("Connection closed.")


In [ ]:
# This creates a table of field names and sample values
import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# List of tables to process
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]


# "station_logs" is the big one. WOuld have the same fields/data as MeterValues data in Splunk.




# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Query to fetch the first few rows from the current table
        query = f"SELECT * FROM {table} LIMIT 10;"
        cursor.execute(query)

        # Fetch the rows
        rows = cursor.fetchall()
        # Fetch the column headers
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame from the fetched data
        df = pd.DataFrame(rows, columns=column_names)

        # Prepare the transposed DataFrame
        transposed_data = {
            'Header': column_names,
            'Data Type': [df[col].dtype.name for col in column_names],  # Get the data type
            'Example': [df[col].iloc[0] if not df[col].empty else None for col in column_names]  # Example from the first row
        }

        df_transposed = pd.DataFrame(transposed_data)

        # Write the DataFrame to CSV
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/{table}_fields.csv'
        df_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")

In [ ]:
# This creates a table of sample records

import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Function to fetch non-transposed data
def fetch_non_transposed_data(cursor, table):
    query = f"SELECT * FROM {table} LIMIT 10;"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=column_names)
    return df

# List of tables to process
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Fetch non-transposed data
        df_non_transposed = fetch_non_transposed_data(cursor, table)

        # Write the DataFrame to CSV with new naming convention
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/{table}_example_data.csv'
        df_non_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")


###Create a table for all property info

In [ ]:
import pandas as pd
import os

# File paths
properties_file = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/properties.csv'
property_types_file = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/property_types.csv'
output_file = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/property_lookup_2.csv'

# Load and verify files
if not os.path.exists(properties_file):
    raise FileNotFoundError(f"File not found: {properties_file}")
if not os.path.exists(property_types_file):
    raise FileNotFoundError(f"File not found: {property_types_file}")

properties = pd.read_csv(properties_file)
property_types = pd.read_csv(property_types_file)

# Normalize column names to lowercase and strip whitespace
properties.columns = properties.columns.str.strip().str.lower()
property_types.columns = property_types.columns.str.strip().str.lower()

# Perform the left join with suffixes
property_lookup = properties.merge(
    property_types,
    how='left',  # Use 'left' join to keep all rows from properties and add property_type name where available
    left_on='property_type',  # Assuming 'property_type' is the column in properties.csv
    right_on='id',  # Assuming 'id' is the column in property_types.csv
    suffixes=('_property', '_type')
)

# Keep all columns from properties and just add the 'name_type' column as 'property_type'
property_lookup['property_type'] = property_lookup['name_type']

# Drop the 'name_type' column, since we already added it as 'property_type'
property_lookup = property_lookup.drop(columns=['name_type'])

# Rename 'id_property' column to 'property_id'
property_lookup = property_lookup.rename(columns={'id_property': 'property_id'})

# Save the resulting DataFrame to CSV
property_lookup.to_csv(output_file, index=False)
print(f"Property lookup table saved to {output_file}")


# Now I need to build the correct table directly from RS

In [ ]:
import os
import pandas as pd
import logging
from itertools import combinations

# Configure logging
logging.basicConfig(level=logging.INFO)

# Path to the directory containing the CSV files
data_dir = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/'

# List of tables (as per your previous code)
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]

# Function to load CSV files into DataFrames
def load_dataframes(tables):
    dataframes = {}
    for table in tables:
        csv_path = os.path.join(data_dir, f"{table}_example_data.csv")
        try:
            df = pd.read_csv(csv_path)
            dataframes[table] = df
            logging.info(f"Loaded data for table: {table}")
        except Exception as e:
            logging.error(f"Error loading data for table {table}: {e}")
    return dataframes

# Function to find strict join matches
def find_strict_joins(df1, df2, table1_name, table2_name):
    strict_joins = []
    # Iterate over all column pairs
    for col1 in df1.columns:
        for col2 in df2.columns:
            if df1[col1].dtype == df2[col2].dtype:
                # Perform the join
                joined_df = pd.merge(df1, df2, left_on=col1, right_on=col2, how='inner')
                # Check if all rows in df1 are in the joined DataFrame
                if len(joined_df) == len(df1):
                    strict_joins.append((col1, col2))
                    logging.info(f"Strict join success: {table1_name}.{col1} <-> {table2_name}.{col2}")
    return strict_joins

# Main function to perform the strict join analysis
def analyze_strict_joins(tables):
    dataframes = load_dataframes(tables)
    results = {}
    table_pairs = combinations(tables, 2)

    for table1, table2 in table_pairs:
        df1 = dataframes.get(table1)
        df2 = dataframes.get(table2)

        if df1 is not None and df2 is not None:
            logging.info(f"Analyzing strict joins between {table1} and {table2}")
            joins = find_strict_joins(df1, df2, table1, table2)
            if joins:
                results[f"{table1} <-> {table2}"] = joins
        else:
            logging.warning(f"Data for {table1} or {table2} is missing. Skipping.")

    return results

# Run the strict join analysis
strict_join_results = analyze_strict_joins(tables)

# Print the results
for table_pair, joins in strict_join_results.items():
    print(f"\nStrict joins for {table_pair}:")
    for col1, col2 in joins:
        print(f"Columns: {table_pair.split(' <-> ')[0]}.{col1} <-> {table_pair.split(' <-> ')[1]}.{col2}")

if not strict_join_results:
    print("No strict joins found.")


In [ ]:
import os
import logging
import pandas as pd
from sqlalchemy import create_engine

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection string for SQLAlchemy
connection_string = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(connection_string)

# Function to fetch column names for a table
def get_columns(table_name):
    try:
        query = f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table_name}';
        """
        with engine.connect() as connection:
            df = pd.read_sql_query(query, connection)
        return df[['column_name', 'data_type']].to_dict('records')
    except Exception as e:
        logging.error(f"Error fetching columns for table {table_name}: {e}")
        return []

# Function to test join logic between two tables
def test_joins(table1, table2, attempts=3):
    columns_table1 = get_columns(table1)
    columns_table2 = get_columns(table2)
    successful_joins = []

    for col1 in columns_table1:
        for col2 in columns_table2:
            # Only test joins on matching data types
            if col1['data_type'] == col2['data_type']:
                success_count = 0
                for _ in range(attempts):  # Attempt the join multiple times
                    query = f"""
                    SELECT *
                    FROM {table1} t1
                    INNER JOIN {table2} t2
                    ON t1.{col1['column_name']} = t2.{col2['column_name']}
                    LIMIT 1;  -- Test with one row at a time
                    """
                    try:
                        with engine.connect() as connection:
                            df = pd.read_sql_query(query, connection)
                            if not df.empty:
                                success_count += 1
                    except Exception as e:
                        logging.debug(f"Join failed for {table1}.{col1['column_name']} = {table2}.{col2['column_name']}: {e}")

                if success_count == attempts:  # Only count as successful if all attempts work
                    successful_joins.append((col1['column_name'], col2['column_name']))
                    logging.info(f"Successful join: {table1}.{col1['column_name']} = {table2}.{col2['column_name']}")

    return successful_joins

# Cross-check join fields for all table pairs
tables = [
    "users", "ocpp_sub_session"
]

results = {}

for i, table1 in enumerate(tables):
    for table2 in tables[i+1:]:
        logging.info(f"Testing joins between {table1} and {table2}")
        joins = test_joins(table1, table2)
        if joins:
            results[f"{table1} <-> {table2}"] = joins
        else:
            logging.info(f"No join found between {table1} and {table2}")

# Print results
for table_pair, joins in results.items():
    print(f"Successful joins for {table_pair}: {joins}")

if not results:
    print("No successful joins found.")


In [ ]:
import pandas as pd
import json

# Example dataframe (assuming df['message'] contains the raw strings)
# Clean the 'message' column by removing the prefix 'OCPP : MeterValues '
def clean_message(msg):
    try:
        # Remove the prefix
        msg_cleaned = msg.lstrip('OCPP : MeterValues ')

        # Attempt to load the cleaned message as JSON
        return json.loads(msg_cleaned)
    except (json.JSONDecodeError, TypeError):
        # If the message cannot be decoded as JSON, return None or handle as needed
        return None

# Apply the function to the 'message' column
df['message'] = df['message'].apply(clean_message)

# Filter out rows where the 'message' column is None (indicating a JSON parse failure)
df = df[df['message'].notna()]

# Step 1: Extract top-level fields and keep 'meterValue' as is (as a list of dicts)
flattened_rows = []

for idx, row in df.iterrows():
    message = row['message']  # Now this is a valid JSON object

    # Extract top-level fields
    connector_id = message.get('connectorId')
    transaction_id = message.get('transactionId')

    # Keep the 'meterValue' field as is (as a list of dicts)
    meter_value = message.get('meterValue', [])

    # Add a row to the flattened list, including the nested 'meterValue' list
    flattened_rows.append({
        '_time': row['time'],  # Retain the original timestamp from the dataframe
        'user_id': row['user_id'],  # Assuming 'user_id' is part of the original dataframe
        'connectorId': connector_id,
        'meterValue': meter_value  # The entire 'meterValue' field, as it is (list of dictionaries)
    })

# Step 2: Create a new DataFrame from the flattened rows
flattened_df = pd.DataFrame(flattened_rows)

# Display the resulting DataFrame
print(flattened_df.head())


In [ ]:
import pandas as pd

# Set pandas options to display the full content of any column (e.g., 'meterValue')
pd.set_option('display.max_colwidth', None)

# Now, display the full content of the 'meterValue' column for the first 5 rows
print(flattened_df['meterValue'].head(1))

In [ ]:
import pandas as pd

# Create a list to hold the expanded rows
expanded_rows = []

# Iterate over each row in the dataframe
for idx, row in flattened_df.iterrows():
    meter_values = row['meterValue']  # This is the list of meter readings (list of dicts)

    # For each meter value entry (there should be one timestamp and a list of measurements)
    for meter in meter_values:
        timestamp = meter['timestamp']  # Extract the timestamp

        # Initialize values for each measurement type
        watt_hours_value = None  # WattHours
        amps_value = None        # Amps (Current)
        voltage_value = None     # Voltage (Volts)

        # Iterate over the sampledValue list (which contains the three measurements)
        for sample in meter['sampledValue']:
            # Check the 'unit' to assign the value to the correct column
            if sample['unit'] == 'Wh':  # WattHours
                watt_hours_value = sample['value']
            elif sample['unit'] == 'A':  # Amps (Current)
                amps_value = sample['value']
            elif sample['unit'] == 'V':  # Volts (Voltage)
                voltage_value = sample['value']

        # Append the expanded row with the extracted values
        expanded_rows.append({
            '_time': row['_time'],  # Retain the original timestamp from the dataframe
            'user_id': row['user_id'],  # Assuming 'user_id' is part of the original dataframe
            'connectorId': row['connectorId'],  # Connector ID
            'timestamp': timestamp,  # Timestamp from the meter value
            'WattHours': watt_hours_value,  # Renamed to WattHours
            'Amps': amps_value,  # Keep Amps as the column name
            'Voltage': voltage_value  # Value for Voltage (V)
        })

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Convert the numeric columns to appropriate types (float)
expanded_df['WattHours'] = pd.to_numeric(expanded_df['WattHours'], errors='coerce')
expanded_df['Amps'] = pd.to_numeric(expanded_df['Amps'], errors='coerce')
expanded_df['Voltage'] = pd.to_numeric(expanded_df['Voltage'], errors='coerce')

# Display the resulting DataFrame
print(expanded_df.head())


In [ ]:
# Ensure all columns are numeric (in case there are any string values left)
expanded_df['WattHours'] = pd.to_numeric(expanded_df['WattHours'], errors='coerce')
expanded_df['Amps'] = pd.to_numeric(expanded_df['Amps'], errors='coerce')
expanded_df['Voltage'] = pd.to_numeric(expanded_df['Voltage'], errors='coerce')

# Classify values as 0 or > 0 for each of the measurements
expanded_df['WattHours_Class'] = expanded_df['WattHours'].apply(lambda x: '0' if x == 0 else '>0')
expanded_df['Amps_Class'] = expanded_df['Amps'].apply(lambda x: '0' if x == 0 else '>0')
expanded_df['Voltage_Class'] = expanded_df['Voltage'].apply(lambda x: '0' if x == 0 else '>0')

# Set up the plot
plt.figure(figsize=(18, 6))

# Plot the count of each class for 'WattHours', 'Amps', and 'Voltage'
plt.subplot(1, 3, 1)
sns.countplot(data=expanded_df, x='WattHours_Class')
plt.title('Count of Rows with WattHours: 0 vs > 0')
plt.xlabel('WattHours Class')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
sns.countplot(data=expanded_df, x='Amps_Class')
plt.title('Count of Rows with Amps: 0 vs > 0')
plt.xlabel('Amps Class')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
sns.countplot(data=expanded_df, x='Voltage_Class')
plt.title('Count of Rows with Voltage: 0 vs > 0')
plt.xlabel('Voltage Class')
plt.ylabel('Count')

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure that the '_time' column is in datetime format
expanded_df['timestamp'] = pd.to_datetime(expanded_df['timestamp'], errors='coerce')

# Convert 'Amps', 'WattHours', and 'Voltage' to numeric (handling any errors)
expanded_df['Amps'] = pd.to_numeric(expanded_df['Amps'], errors='coerce')
expanded_df['WattHours'] = pd.to_numeric(expanded_df['WattHours'], errors='coerce')
expanded_df['Voltage'] = pd.to_numeric(expanded_df['Voltage'], errors='coerce')

# Drop rows where any of the values are missing
expanded_df = expanded_df.dropna(subset=['_time', 'Amps', 'WattHours', 'Voltage'])

# Set the style for the plots
sns.set(style="whitegrid")

# Create the figure and axes for the plots
plt.figure(figsize=(18, 6))

# Plot Amps over time
plt.subplot(1, 3, 1)
plt.plot(expanded_df['timestamp'], expanded_df['Amps'], label='Amps', color='b', alpha=0.7)
plt.title('Amps over Time')
plt.xlabel('Time')
plt.ylabel('Amps')
plt.xticks(rotation=45)

# Plot WattHours over time
plt.subplot(1, 3, 2)
plt.plot(expanded_df['timestamp'], expanded_df['WattHours'], label='WattHours', color='g', alpha=0.7)
plt.title('WattHours over Time')
plt.xlabel('Time')
plt.ylabel('WattHours')
plt.xticks(rotation=45)

# Plot Voltage over time
plt.subplot(1, 3, 3)
plt.plot(expanded_df['timestamp'], expanded_df['Voltage'], label='Voltage', color='r', alpha=0.7)
plt.title('Voltage over Time')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.xticks(rotation=45)

# Adjust layout to avoid overlap of labels
plt.tight_layout()
plt.show()


In [ ]:
expanded_df.info()

In [ ]:
# Run descriptive statistics on 'Amps', 'WattHours', and 'Voltage'
descriptive_stats = expanded_df[['Amps', 'WattHours', 'Voltage']].describe()

# Display the statistics
print(descriptive_stats)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for the plots
sns.set(style="whitegrid")

# Create a figure with 3 subplots (1 row, 3 columns)
plt.figure(figsize=(18, 6))

# Plot for Amps
plt.subplot(1, 3, 1)
sns.boxplot(data=expanded_df['Amps'], color='skyblue')
plt.title('Boxplot of Amps')
plt.ylabel('Amps')

# Plot for WattHours
plt.subplot(1, 3, 2)
sns.boxplot(data=expanded_df['WattHours'], color='lightgreen')
plt.title('Boxplot of WattHours')
plt.ylabel('WattHours')

# Plot for Voltage
plt.subplot(1, 3, 3)
sns.boxplot(data=expanded_df['Voltage'], color='lightcoral')
plt.title('Boxplot of Voltage')
plt.ylabel('Voltage')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()


In [ ]:
# count of propertyIDs

import os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'property_id' values
query = f"SELECT COUNT(DISTINCT property_id) FROM {table};"
cursor.execute(query)

# Fetch the result
result = cursor.fetchone()

# Extract and print the count
unique_property_id_count = result[0] if result else 0
print(f"Number of unique 'property_id' values: {unique_property_id_count}")

# Clean up
cursor.close()
connection.close()

In [ ]:
#Count of cluster IDs

import os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'property_id' values
query = f"SELECT COUNT(DISTINCT cluster_id) FROM {table};"
cursor.execute(query)

# Fetch the result
result = cursor.fetchone()

# Extract and print the count
unique_property_id_count = result[0] if result else 0
print(f"Number of unique 'cluster_id' values: {unique_property_id_count}")

# Clean up
cursor.close()
connection.close()

In [ ]:
# counts of peropertyID and clusterIDimport os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'cluster_id' values for each 'property_id'
query = f"""
SELECT property_id, COUNT(DISTINCT cluster_id)
FROM {table}
GROUP BY property_id
;
"""
cursor.execute(query)

# Fetch all the results
results = cursor.fetchall()

# Print the results
for row in results:
    property_id, cluster_count = row
    print(f"Property ID: {property_id}, Unique Cluster ID Count: {cluster_count}")

# Clean up
cursor.close()
connection.close()

import os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'property_id' values for each 'cluster_id'
query = f"""
SELECT cluster_id, COUNT(DISTINCT property_id)
FROM {table}
GROUP BY cluster_id;
"""
cursor.execute(query)

# Fetch all the results
results = cursor.fetchall()

# Print the results
for row in results:
    cluster_id, property_count = row
    print(f"Cluster ID: {cluster_id}, Unique Property ID Count: {property_count}")

# Clean up
cursor.close()
connection.close()


OCPP_SessionID has a userID and TransactionID
I need to map to the cluster and property

Stations has propertyID and cluster_id




In [ ]:
# Clean message field and port to a df
import json
import pandas as pd

# Function to clean up the 'message' field by removing the prefix and parsing JSON
def clean_and_parse_message(message):
    try:
        # Strip the non-JSON prefix before the first '{'
        cleaned_message = message[message.find('{'):]
        # Parse the cleaned JSON string
        return json.loads(cleaned_message)
    except json.JSONDecodeError as e:
        print(f"Invalid JSON in message: {message}\nError: {e}")
        return None

# Function to flatten nested JSON
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], name + a + '_')
        elif isinstance(x, list):
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Apply the cleaning and parsing function to all rows in the 'message' field
df['parsed_message'] = df['message'].apply(clean_and_parse_message)

# Drop rows where parsing failed (invalid JSON) or was not cleaned properly
valid_df = df[df['parsed_message'].notnull()]

# Flatten all the JSON objects and store them in a new DataFrame
flattened_data = valid_df['parsed_message'].apply(flatten_json).apply(pd.Series)

# Combine the flattened JSON fields with the original DataFrame (excluding the original 'message' and 'parsed_message' fields)
new_df = pd.concat([valid_df.drop(columns=['message', 'parsed_message']), flattened_data], axis=1)

# Write the DataFrame to CSV with new naming convention
output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/new_df.csv'
new_df.to_csv(output_csv_path, index=False)
logging.info(f"Data written to {output_csv_path} successfully.")




In [ ]:
new_df.info
new_df.head()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df= new_df

# Assuming your DataFrame is named df
# Step 1: Convert 'time' to datetime
df['time'] = pd.to_datetime(df['time'], errors='coerce')  # errors='coerce' will turn invalid parsing to NaT

# Step 2: Convert 'meterValue_0_sampledValue_0_value' to numeric
df['meterValue_0_timestamp'] = pd.to_numeric(df['meterValue_0_timestamp'], errors='coerce')

# Step 3: Drop any rows with NaT or NaN values (optional, depending on your needs)
df = df.dropna(subset=['time', 'meterValue_0_sampledValue_0_value'])

# Step 4: Plot the time series
plt.figure(figsize=(10, 6))
plt.plot(df['meterValue_0_timestamp'], df['meterValue_0_sampledValue_0_value'], label='Meter Value', color='b')
plt.xlabel('Time')
plt.ylabel('Meter Value')
plt.title('Meter Value Over Time')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# Group by user_id and count the occurrences of meterValue_0_timestamp
hist_data = df.groupby('user_id')['message'].count().reset_index()

# Rename the columns for clarity
hist_data.columns = ['user_id', 'count']

# Sort the data by user_id for better visualization
hist_data = hist_data.sort_values('user_id')

# Plotting the normal line plot
plt.figure(figsize=(10, 6))
plt.plot(hist_data['user_id'], hist_data['count'], marker='o', linestyle='-', color='skyblue')
plt.xlabel('User ID')
plt.ylabel('Count of message')
plt.title('Count of message per User ID')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
import pandas as pd

# Assuming new_df is already defined and contains the necessary columns

# List of columns to keep
columns_to_keep = [
    #'time',
    'user_id',
    #'station_id',
    'property_id',
    'connectorId',
    'meterValue_0_timestamp',
    'meterValue_0_sampledValue_1_value',
    'meterValue_0_sampledValue_1_context',
    'meterValue_0_sampledValue_1_format',
    'meterValue_0_sampledValue_1_measurand',
    'meterValue_0_sampledValue_1_phase',
    'meterValue_0_sampledValue_1_location',
    'meterValue_0_sampledValue_1_unit'
]

# Create new_df_2 with only the selected columns
new_df_2 = new_df[columns_to_keep].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Convert 'time' to datetime
new_df_2['meterValue_0_timestamp'] = pd.to_datetime(new_df_2['meterValue_0_timestamp'], errors='coerce')

# Check for any NaT values that may have resulted from the conversion
if new_df_2['meterValue_0_timestamp'].isnull().any():
    print("Some values could not be converted to datetime.")

# Extract day and hour using .loc to avoid warnings
new_df_2.loc[:, 'meterValue_0_day'] = new_df_2['meterValue_0_timestamp'].dt.date
new_df_2.loc[:, 'meterValue_0_hour'] = new_df_2['meterValue_0_timestamp'].dt.hour



In [ ]:
new_df_2.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = new_df_2

# Assuming df is your DataFrame
# Group by user_id and count the occurrences of meterValue_0_timestamp
hist_data = df.groupby('user_id')['meterValue_0_timestamp'].count().reset_index()

# Rename the columns for clarity
hist_data.columns = ['user_id', 'count']

# Sort the data by user_id for better visualization
hist_data = hist_data.sort_values('user_id')

# Plotting the normal line plot
plt.figure(figsize=(10, 6))
plt.plot(hist_data['user_id'], hist_data['count'], marker='o', linestyle='-', color='skyblue')
plt.xlabel('User ID')
plt.ylabel('Count of message')
plt.title('Count of message per User ID')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
# Assuming new_df_2 is the df

unique_values = new_df_2['user_id'].unique()

# To display the unique values
print(unique_values)


# Assuming new_df_2 is your DataFrame
unique_count = new_df_2['user_id'].nunique()

# To display the count of unique user_id values
print(f"Number of unique user_id values: {unique_count}")

In [ ]:
import pandas as pd


new_df_2['meterValue_0_sampledValue_1_value'] = pd.to_numeric(new_df_2['meterValue_0_sampledValue_1_value'], errors='coerce')

max_values = new_df_2.loc[new_df_2.groupby(['user_id', 'meterValue_0_day'])['meterValue_0_sampledValue_1_value'].idxmax()]

result_df = max_values[['user_id', 'meterValue_0_day', 'meterValue_0_sampledValue_1_value', 'meterValue_0_timestamp']]

print(result_df)
result_df.info()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# Group by user_id and count the occurrences of meterValue_0_timestamp
hist_data = df.groupby('user_id')['meterValue_0_timestamp'].count().reset_index()

# Rename the columns for clarity
hist_data.columns = ['user_id', 'count']

# Sort the data by user_id for better visualization
hist_data = hist_data.sort_values('user_id')

# Plotting the normal line plot
plt.figure(figsize=(10, 6))
plt.plot(hist_data['user_id'], hist_data['count'], marker='o', linestyle='-', color='skyblue')
plt.xlabel('User ID')
plt.ylabel('Count of meterValue_0_timestamp')
plt.title('Count of meterValue_0_timestamp per User ID')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming result_df is your DataFrame

# Print column names to verify
print("Column names in DataFrame:")
print(result_df.columns.tolist())

# Specify the user_id you're interested in
specific_user_id = '013f0335-da69-4fdd-b378-b6a9a8cfc8a8'  # replace with the actual user_id

# Filter the DataFrame for the specific user_id
filtered_df = result_df[result_df['user_id'] == specific_user_id]

# Check if there are any rows for the specified user_id
if not filtered_df.empty:
    # Check for the timestamp column again
    timestamp_col = 'meterValue_0_timestamp'  # Update if necessary
    value_col = 'meterValue_0_sampledValue_1_value'

    # Ensure the column names are correct
    print("Filtered DataFrame columns:")
    print(filtered_df.columns.tolist())

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(filtered_df[timestamp_col], filtered_df[value_col], marker='o')
    plt.title(f'Meter Values for User ID: {specific_user_id}')
    plt.xlabel('Timestamp')
    plt.ylabel('Meter Value')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
    plt.grid()
    plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
    plt.show()
else:
    print(f"No data found for user_id: {specific_user_id}")


In [ ]:

# Write the DataFrame to CSV
output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/result_df_exported.csv'
result_df.to_csv(output_csv_path, index=False)
logging.info(f"Data written to {output_csv_path} successfully.")


